In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import FirefoxOptions
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep #스크롤다운을 위한 대기시간 sleep
from pyspark.sql import SparkSession
from pyspark import SparkContext
import requests
import json
from pyspark.sql.functions import *
import requests
import json
from urllib import parse
from urllib.request import urlopen
from urllib.request import Request
from urllib.error import HTTPError
import json
from pyspark.sql import Row

In [ ]:
spark = SparkSession.builder.appName("idus_final").getOrCreate()

In [ ]:
#1차크롤링(아이디어스에 등록되어있는 상점명만가져오기)
base_url = "https://www.idus.com/c/region/101"
webdriver_service = Service('/home/big/DE_Pr/idus/geckodriver')

opts = FirefoxOptions()
opts.add_argument("--headless")		
browser = webdriver.Firefox(service=webdriver_service, options=opts)

#스크롤을내림에 따라 list가증가하니까, 스크롤을 먼저내려줌

SCROLL_PAUASE_TIME=0.8

browser.get(base_url)

In [ ]:
while True:
    sleep(SCROLL_PAUASE_TIME)
    #스크롤을 내려준다
    last_height = browser.execute_script("return document.body.scrollHeight")
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(SCROLL_PAUASE_TIME)
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        browser.execute_script(
            "window.scrollTo(0, document.body.scrollHeight);")
        sleep(SCROLL_PAUASE_TIME)
        new_height = browser.execute_script(			
            "return document.body.scrollHeight")
        if new_height == last_height:
            break
        else:
            last_height = new_height
            continue

#DATA-TAGET-ID만 추출하여 리스트안에 넣어놓자
classid=browser.find_elements(By.CLASS_NAME,'ui_card--white')

classid_copy=[]
for comment in classid:
    classid_copy.append(comment.get_attribute('data-target-id'))

classid_list=classid_copy

print('1.scesss')

In [ ]:
final_url=[]
no_url=[]
num=1
for class_name in classid_list:
    class_url=f"https://www.idus.com/c/class/{class_name}"
    browser.get(class_url)
    try:
        sleep(1.5)
        link1=browser.find_element(By.CSS_SELECTOR,'div.artist_card__split a').click()
        link2=browser.find_element(By.CSS_SELECTOR,'div.ui_grid__item a').get_attribute('href')
        final_url.append(link2)
        num=num+1
        print(num,'scess')
        #print(final_url)
    except NoSuchElementException as ns:
        no_url_name=browser.find_element(By.CLASS_NAME,'artist-info__name').text
        no_url.append(no_url_name)
        num=num+1
        print(num,'scess')

## 최초분기지점 (주소 포함, 미포함)

In [ ]:
#수집(2)

final_url_dist = list(dict.fromkeys(final_url))
no_url_dist=list(dict.fromkeys(no_url))

print(len(final_url)) 
print(len(final_url_dist)) # 주소 포함된
print(len(no_url))
print(len(no_url_dist)) # 주소 미포함

info_dict={}
info_list=[]
no_info_list=[]


for final_urls in final_url_dist:
    browser.get(final_urls)
    try:
        artist_name=browser.find_element(By.CLASS_NAME,'artist_card__label').text.replace('\n','')
        activetab=browser.find_element(By.XPATH,"//*[@data-ui-id='info-artist']").click()
        store_name=browser.find_element(By.XPATH,"//*[@data-panel-id='info-artist']/tbody/tr[1]/td").text.replace('\n','')
        address=browser.find_element(By.XPATH,"//*[@data-panel-id='info-artist']/tbody/tr[3]/td").text.replace('\n','')
        sellernum=browser.find_element(By.XPATH,"//*[@data-panel-id='info-artist']/tbody/tr[4]/td").text.replace('\n','')#번호는 굳이 필요없지만, 가져오지 않을 리스트를 만들기위해 일부러 오류
        info_list.append([artist_name,store_name,address])
    except NoSuchElementException as e:
        artist_name=browser.find_element(By.CLASS_NAME,'artist_card__label').text.replace('\n','')
        no_info_list.append([artist_name])

## 주소 포함된 경우

In [ ]:
#수집(3)

#로컬에만들어짐(idus_b.csv)

# idus홈페이지에서 크롤링한 정보 베이스 csv파일(추후 x,y,경도,위도 추가)
# f = open("idus_b.csv", "w",encoding='utf-8')
# w = csv.writer(f)
# w.writerow(['artist_name','store_name','address'])
# for cont in info_list:
#     w.writerow(cont)  
# f.close()
# info_list.write.format('csv').mode('append').save('/idus_final/idus_b.csv')

# sava csv as hdfs df

local_creator_index = ["artist_name", "store_name", "address"]
loc_creator = spark.createDataFrame(info_list, local_creator_index)
loc_creator.createOrReplaceTempView("loc_creator")
loc_creator.write.options(header='True', delimiter=',', encoding="utf-8").csv("/local_creator/idus_b.csv")

### load data

In [ ]:
idus_b = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/local_creator/idus_b.csv")
)

In [ ]:
idus_b.show()

In [ ]:
init_addr = idus_b.select('address').rdd.map(lambda x:x[0]).collect() # 주소 포함
init_artist_name = idus_b.select('artist_name').rdd.map(lambda x:x[0]).collect() # 주소 포함
init_store_name = idus_b.select('store_name').rdd.map(lambda x:x[0]).collect() # 주소 포함

### generate coordinate

In [ ]:
market_coor =[]
for i in init_addr:
    Client_ID = "0dahjnik2r"
    Client_Secret="laJOj2DMSCX29PCCXMY6thLXVHtavtomJ1qDQT6t"
    api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='
    test = i # 주소는 2번째 위치 >> [1]
    try:
        add_urlenc = parse.quote(test) # URL Encoding
    except:
        # trick. 변환 불가할 경우 0,0 좌표를 줘서 1km 필터에 걸리지 않도록 처리
        latitude, longitude = 0, 0
    print(add_urlenc)
    url = api_url + add_urlenc
    request = Request(url)   
    request.add_header('X-NCP-APIGW-API-KEY-ID', Client_ID)
    request.add_header('X-NCP-APIGW-API-KEY', Client_Secret)

    try:
        response = urlopen(request)

    except HTTPError as e:
        print('HTTP Error')
        latitude, longitude = 0, 0

    else:
        rescode = response.getcode()
        
        
        if rescode == 200:
            coordset = []
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)
            
            if response_body['addresses'] == []:
                coordset.append(0)
                coordset.append(0)
                market_coor.append(coordset)
            else:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
                print('Success')
                print(latitude, longitude)
                coordset.append(latitude)
                coordset.append(longitude)
                market_coor.append(coordset)
        else:
            print(f'Response error, rescode:{rescode}')
            latitude, longitude = 0, 0
            coordset.append(latitude)
            coordset.append(longitude)
            market_coor.append(coordset)

In [ ]:
# result check
print(init_store_name[0])
print((init_artist_name[0]))
print((init_addr[0]))
print((market_coor[0][0]))

In [ ]:
# api 변환 불가지점 확인
pop_index_list = []
for i in range(len(init_addr)):
    if market_coor[i][0] ==0:
        pop_index_list.append(i)
        print(init_addr[i])
# 변환 불가지점 좌표 전환 확인
for i in pop_index_list:
    print((init_addr)[i])
    print((market_coor[i]))

### make dataframe

In [ ]:
initial_index = ['artist_name','store_name','address','lot','lat']
temp = []
for i in range(len(init_artist_name)):
    count = Row(init_artist_name[i], init_store_name[i], init_addr[i], float(market_coor[i][0]), float(market_coor[i][1]))
    temp.append(count)


In [ ]:
c_store_initial_df = spark.createDataFrame(temp, initial_index)
c_store_initial_df.write.options(header='True', delimiter=',', encoding="utf-8").csv("c_store_initial_df.csv")

### Initial SAVEPOINT

In [ ]:
load_test = spark.read.option("header", "true").csv("c_store_initial_df.csv")
load_test.show()

## 주소 포함되지 않은 경우
* 결측치 f
* 미측정값 no_info_list

In [ ]:
# 결측치 확인과정

#print((info_list))
#print((no_info_list))
for i in info_list:
    if i[2] == "":
        print(i)


In [ ]:
# 전처리(1)-spark

# 위에서 생성한 베이스csv파일을 열어서 주소가없는 셀러 추출하여저장


# idus_b.printSchema()
# idus_b.show(idus_b.count())
idus_b.createOrReplaceTempView("idus")
# Initial list

# 결측치 다룰 df 생성
info_no_list = spark.sql("""select store_name from idus where address is null""")
info_no_list.write.options(header='True', delimiter=',', encoding="utf-8").csv('/idus_final/info_no_list.csv')


In [ ]:
# 전처리(2)-python

# 다운로드받아야함(info_no_list.csv)
# 예외처리에서 나온 목록들을 검색하기위해 1차원리스트로 다 합쳐준다


f = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/idus_final/info_no_list.csv")
)  # -> info_no_list.write.format('csv').mode('overwrite').option("header","true").save('/na/')



In [ ]:
f.show()

In [ ]:
print(no_info_list)

In [ ]:
data = f.select('store_name').rdd.map(lambda x:x[0]).collect()
temp = []
for i in no_info_list:
    temp.append(i[0])

no_address_t = (
    data + temp
)  # data(idus_b.csv 에서 나온 info_no_list.csv) + no_info_list(수집(2)에서 제외된 리스트 변수)

result_no_url = []
for no_address_ts in no_address_t:
    result_no_url.extend(no_address_ts)

result_no_url = no_address_t + no_url_dist  # +수집(1)에서 제외된 리스트 변수

In [ ]:
print(result_no_url)

In [ ]:
# 수집(4)-kakao rest api(python)

# 예외처리 목록들을 카카오api로 검색. 정확도가떨어져 수동확인 완료

method = "GET"

kakaoapi_no_url = result_no_url
exception_list = []
for kakaoapi_no_urls in kakaoapi_no_url:

    params = {"query": kakaoapi_no_urls}
    url = "https://dapi.kakao.com/v2/local/search/keyword"

    header = {"Authorization": "KakaoAK 0fda3bab295d6e5f647174eff2af12f7"}
    response = requests.get(url, headers=header, params=params)
    tokens = response.json()

    try:

        tokens_list = list(tokens.values())

        tokens_dict = dict(tokens_list[0][0])

        artist_name = tokens_dict["place_name"]
        address = tokens_dict["address_name"]
        lot = float(tokens_dict["x"])
        lat = float(tokens_dict["y"])

        exception_list.append([artist_name, kakaoapi_no_urls, address, lot, lat])

    except:
        print("PASS : ", kakaoapi_no_urls)


In [ ]:
print((exception_list[0][4]))
print((exception_list[0][3]))

### Exception SAVE POINT

In [ ]:
# save as csv

c_store_exception_index = ['artist_name','store_name','address','lot','lat']
temp2 = []
for i in range(len(exception_list)):
    count = Row(exception_list[i][0], exception_list[i][1], exception_list[i][2], exception_list[i][3], exception_list[i][4])
    temp2.append(count)

In [ ]:
c_store_exception_df = spark.createDataFrame(temp2, c_store_exception_index)
c_store_exception_df.createOrReplaceTempView("exception_df")
c_store_exception_df.write.options(header='True', delimiter=',', encoding="utf-8").csv("c_store_exception_df.csv")

In [ ]:
load_test2 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("c_store_exception_df.csv")
load_test2.createOrReplaceTempView("c_store_exception_df")
load_test2.show()

## integrate initial, exception df

In [ ]:
c_store_initial_df.show()
c_store_exception_df.show()

In [ ]:
integrated = c_store_initial_df.union(c_store_exception_df)

In [ ]:
integrated.show()

## Make Final result

In [ ]:
integrated_name = integrated.select('store_name').rdd.map(lambda x:x[0]).collect()
integrated_lot = integrated.select('lot').rdd.map(lambda x:x[0]).collect()
integrated_lat = integrated.select('lat').rdd.map(lambda x:x[0]).collect()
store_coor = []
for i in range(len(integrated_name)):
    temp = []
    temp.append(integrated_lot[i])
    temp.append(integrated_lat[i])
    store_coor.append(temp)

In [ ]:
market = spark.read.option("header", "true").option("encoding","euc-kr").csv('market.csv')
market.createOrReplaceTempView("market")

In [ ]:
market_x = market.select('x').rdd.map(lambda x:x[0]).collect()
market_y = market.select('y').rdd.map(lambda x:x[0]).collect()
market_name = market.select('TRDAR_NM').rdd.map(lambda x:x[0]).collect()

In [ ]:
market_coor=[]
for i in range(len(market_x)):
    temp=[]
    temp.append(float(market_x[i]))
    temp.append(float(market_y[i]))
    market_coor.append(temp)    

In [ ]:
print(store_coor[-1])
print(market_coor)

In [ ]:
# pip install haversine
from haversine import haversine

final_result = []
for j in range(len(market_coor)):
    temp = []
    temp.append(market_name[j])
    store_count = 0
    for i in range(len(store_coor)):
        point_1 = (store_coor[i][1], store_coor[i][0])
        point_2 = (market_coor[j][1], market_coor[j][0])
        if haversine(point_1, point_2, unit = 'km') < 1:
            store_count += 1
    temp.append(store_count)
    final_result.append(temp)

In [414]:
print(len(final_result))

1090


In [ ]:
final_index = ['TRDAR_NM','culture_store_count']
temp3 = []
for i in range(len(final_result)):
    count = Row(final_result[i][0].replace(",",""),final_result[i][1])
    temp3.append(count)
final_df = spark.createDataFrame(temp3, final_index)

In [ ]:
final_df.show()

### final result savepoint

In [ ]:
final_df.write.options(header='True', delimiter=',', encoding="utf-8").csv("c_store_final_df.csv")

In [ ]:
load_test3 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("c_store_final_df.csv")

In [412]:
#load_test3.show()
asdf = load_test3.select('TRDAR_NM').rdd.map(lambda x:x[0]).collect()
print(len(asdf))

27


2022-12-08 23:25:40,868 INFO datasources.FileSourceStrategy: Pushed Filters: 
2022-12-08 23:25:40,868 INFO datasources.FileSourceStrategy: Post-Scan Filters: 
2022-12-08 23:25:40,868 INFO datasources.FileSourceStrategy: Output Data Schema: struct<TRDAR_NM: string>
2022-12-08 23:25:40,869 INFO memory.MemoryStore: Block broadcast_551 stored as values in memory (estimated size 303.0 KiB, free 433.7 MiB)
2022-12-08 23:25:40,877 INFO storage.BlockManagerInfo: Removed broadcast_550_piece0 on ubuntu:42485 in memory (size: 6.8 KiB, free: 434.3 MiB)
2022-12-08 23:25:40,881 INFO memory.MemoryStore: Block broadcast_551_piece0 stored as bytes in memory (estimated size 53.8 KiB, free 433.7 MiB)
2022-12-08 23:25:40,881 INFO storage.BlockManagerInfo: Added broadcast_551_piece0 in memory on 192.168.254.129:43935 (size: 53.8 KiB, free: 434.3 MiB)
2022-12-08 23:25:40,881 INFO spark.SparkContext: Created broadcast 551 from javaToPython at <unknown>:0
2022-12-08 23:25:40,881 INFO execution.FileSourceScanE